In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns

import warnings
warnings.filterwarnings('ignore') # setting ignore as a parameter

In [2]:
#Reading the data from reduced files
#df_ch_f = pd.read_pickle('data/df_channels_reduced.tsv.gz')
#df_sb_f = pd.read_pickle('data/df_timeseries_reduced.tsv.gz')

#df_sb_f["datetime"] = pd.to_datetime(df_sb_f["datetime"])
#df_ch_f["join_date"] = pd.to_datetime(df_ch_f["join_date"])
columns = ['categories', 'channel_id', 'crawl_date', 'description',
       'dislike_count', 'display_id', 'duration', 'like_count', 'tags',
       'title', 'upload_date', 'view_count']
vd_frames = []
for chunk in tqdm(pd.read_csv('data/yt_metadata_reduced.tsv.gz', chunksize=10**5, names=columns,lineterminator='\n')):
       chunk["dummmy"] = 1
       vd_frames.append(chunk)
df_vd_f = pd.concat(vd_frames)
df_vd_f = df_vd_f[1:]

166it [03:28,  1.26s/it]


In [3]:
#df_vd_f["description"].astype(str)

test_set = df_vd_f[df_vd_f["categories"] == "News & Politics"]




In [4]:
test_set = df_vd_f["description"].values.tolist()

test_text = []
for sent in test_set: 
    test_text.append(str(sent))



In [5]:
import torch

In [ ]:
train_df = pd.read_fwf("data/keyword-tweets.txt")

train_df = pd.read_csv('data/keyword-tweets.txt', delimiter = "\t")

train_df.columns = ["label", "Tweet"]

train_df = train_df.applymap(str)

train_df.label = (train_df["label"] == "POLIT") * 1


In [ ]:
from sklearn.model_selection import train_test_split

train_text = train_df["Tweet"].values.tolist()
train_label = train_df["label"].values.tolist()

train_texts, val_texts, train_labels, val_labels = train_test_split(train_text, train_label, test_size=0.3)



In [ ]:
#from transformers import RobertaTokenizer

#tokenizer = RobertaTokenizer.from_pretrained('roberta-base', add_prefix_space=True, use_fast=False)

#x = tokenizer("Lego toys, Lego cartoons, Lego designers really like kids all over the world. See a lot of cartoons about cars and Lego cartoons, cartoons about Lego police cars, cartoons about Lego trains, cartoons about Lego fire trucks on our channel.", truncation=True, padding=True, add_special_tokens = True, is_split_into_words=True)

In [ ]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', add_prefix_space=True, use_fast=False)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, add_special_tokens = True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, add_special_tokens = True)

for sent in test_text: 
    print(sent)
    test_encodings = tokenizer(test_text, truncation=True, padding=True, add_special_tokens = True, is_split_into_words=True)


loading file vocab.json from cache at C:\Users\Esraa/.cache\huggingface\hub\models--roberta-base\snapshots\ff46155979338ff8063cdad90908b498ab91b181\vocab.json
loading file merges.txt from cache at C:\Users\Esraa/.cache\huggingface\hub\models--roberta-base\snapshots\ff46155979338ff8063cdad90908b498ab91b181\merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at C:\Users\Esraa/.cache\huggingface\hub\models--roberta-base\snapshots\ff46155979338ff8063cdad90908b498ab91b181\config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_

In [ ]:
class make_torch_dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=[]):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if len(self.labels):
            item["labels"] = torch.tensor(self.labels[idx]).to(torch.int64)
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = make_torch_dataset(train_encodings, train_labels)
val_dataset = make_torch_dataset(val_encodings, val_labels)
#test_dataset = make_torch_dataset(test_encodings)


In [ ]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def compute_metrics(p):    
    pred, labels = p
    print(p)
    print(pred)
    print(labels)
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1} 


training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    eval_steps=25,
    evaluation_strategy="steps",
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=50,                # number of warmup steps for learning rate scheduler
    save_steps=100,
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=20,
    seed=0,
    load_best_model_at_end=True,
)

model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    compute_metrics=compute_metrics,
    eval_dataset=val_dataset             # evaluation dataset
)

# Train pre-trained model
trainer.train()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

  0%|          | 0/264 [00:00<?, ?it/s]

{'loss': 0.5686, 'learning_rate': 2e-05, 'epoch': 0.23}


***** Running Evaluation *****
  Num examples = 601
  Batch size = 16


  0%|          | 0/38 [00:00<?, ?it/s]

...

  0%|          | 0/38 [00:00<?, ?it/s]

[[-2.5884979  2.3067703]
 [-2.4605017  2.1203732]
 [-2.4289482  2.0317645]
 ...
 [-2.5161004  2.1976557]
 [-2.4091423  1.972623 ]
 [-2.4789615  2.134314 ]]
[1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1
 0 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 0 1 1 0 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1
 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1 0 1 1 1
 0 1 1 1 1 1 1 0 0 1 1 0 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 



Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results\checkpoint-200 (score: 0.3273032605648041).


{'train_runtime': 9159.2883, 'train_samples_per_second': 0.459, 'train_steps_per_second': 0.029, 'train_loss': 0.34565452779784345, 'epoch': 3.0}


TrainOutput(global_step=264, training_loss=0.34565452779784345, metrics={'train_runtime': 9159.2883, 'train_samples_per_second': 0.459, 'train_steps_per_second': 0.029, 'train_loss': 0.34565452779784345, 'epoch': 3.0})